In [43]:
import scipy.interpolate as interp

In [24]:
df = performance_dataframe()
df_sorted = df.loc[(slice(None), "average_three_dim"), :].reset_index(drop=True).drop(["p_0", "p_1", "p_2"], axis=1)

In [41]:
grid_probs = df_sorted.loc[:, ["p_0_true", "p_1_true", "p_2_true"]].to_numpy()
work_grid = np.array([[0, 0, 0]])
values = np.array([])
for omega in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    edit_grid = grid_probs
    edit_grid[:, 2] = omega
    work_grid = np.append(work_grid, grid_probs, axis=0)
    values = np.append(values, df_sorted.loc[:, omega].to_numpy(), axis=0)
grid = work_grid[1:, :]

In [99]:
def create_eval_prob_omega_grid(prob_gridsize, omega_gridsize):
    vectors = []
    x = y = np.arange(0.01, 0.91, prob_gridsize)
    omega_grid = np.arange(0, 0.5, omega_gridsize)
    for omega in omega_grid:
        for p_1 in x:
            for p_2 in y:
                if p_1 == 0 or p_2 == 0:
                    pass
                elif p_1 + p_2 >= 1:
                    pass
                else:
                    vectors += [[p_1, p_2, omega]]
    return np.array(vectors).round(2)

In [100]:
eval_grid = create_eval_prob_omega_grid(0.01, 0.05)

In [97]:
interp.griddata(grid, values, eval_grid, method="linear")

array([-83859.99010979, -83644.06614183, -83428.14217387, ...,
                   nan,             nan,             nan])